In [9]:
from operator import itemgetter
from itertools import islice

import pandas as pd
import numpy as np

from gensim.models import Word2Vec

# restore objects and unpack them into variables
%store -r object_keep
df_bbc, list_categories, X, y, X_train, X_test, y_train, y_test = itemgetter('df_bbc',
                                                                             'list_categories',
                                                                             'X',
                                                                             'y',
                                                                             'X_train',
                                                                             'X_test',
                                                                             'y_train',
                                                                             'y_test')(object_keep)

## Logistic Regression with Word Embeddings
Thus far, have been rudimentarily counting words. Can compute word embeddings to get the relatedness of words. The point of *word embeddings* is that you can start considering the contexts of words more because you are seeking to understand words by the other words that it is surrounded by. In this regard, *word embeddings* belong to the text pre-processing stage. There are two predominant types we can choose from:
- Word2Vec
- GloVe

Will use the word2vec model by Google which is pre-trained on 100 billion words in the Google News corpus.

In [8]:
from gensim import models.keyedvectors
# load word2vec model
wv = gensim.models.KeyedVectors.load_word2vec_format("../data/GoogleNews-vectors-negative300.bin.gz", binary = True)
wv.init_sims(replace = True)

In [10]:
# explore some vocabularies
list(islice(wv.vocab, 13030, 13050))

['Memorial_Hospital',
 'Seniors',
 'memorandum',
 'elephant',
 'Trump',
 'Census',
 'pilgrims',
 'De',
 'Dogs',
 '###-####_ext',
 'chaotic',
 'forgive',
 'scholar',
 'Lottery',
 'decreasing',
 'Supervisor',
 'fundamentally',
 'Fitness',
 'abundance',
 'Hold']